In [1]:
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [19]:
# OpenWeather API details
API_KEY = 'YOUR_API_KEY'
CITY = 'Kolkata'  
URL = f'http://api.openweathermap.org/data/2.5/forecast?q={CITY}&appid={API_KEY}&units=metric'

In [3]:
# Fetch weather data from OpenWeather API
response = requests.get(URL)
data = response.json()

In [4]:
# Extract the relevant weather features and target variable
weather_data = []
for entry in data['list']:
    weather = {
        'temp': entry['main']['temp'],
        'humidity': entry['main']['humidity'],
        'pressure': entry['main']['pressure'],
        'wind_speed': entry['wind']['speed'],
        'weather_condition': entry['weather'][0]['main'],  # Categorical feature
    }
    weather_data.append(weather)

In [5]:
# Convert to DataFrame
df = pd.DataFrame(weather_data)

In [6]:
# Convert categorical 'weather_condition' to numerical values
df['weather_condition'] = pd.Categorical(df['weather_condition']).codes

In [7]:
# Features and target variable
X = df.drop(['temp'], axis=1)  # Features (Humidity, Pressure, Wind Speed, Weather Condition)
y = df['temp']  # Target (Temperature)

In [8]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
# 1. Linear Regression
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)

In [11]:
# 2. Decision Tree
dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

In [12]:
# 3. Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

In [13]:
# 4. Support Vector Regressor (SVR)
svr = SVR(kernel='rbf')
svr.fit(X_train_scaled, y_train)
y_pred_svr = svr.predict(X_test_scaled)

In [14]:
# 5. K-Nearest Neighbors (KNN)
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train_scaled, y_train)
y_pred_knn = knn.predict(X_test_scaled)

In [15]:
# 6. XGBoost
xgb = XGBRegressor(n_estimators=100, random_state=42)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

In [16]:
# Performance evaluation
def evaluate_model(y_test, y_pred, model_name):
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"\n{model_name} Results:")
    print(f"Mean Squared Error: {mse}")
    print(f"R2 Score: {r2}")

In [17]:
# Evaluate all models
evaluate_model(y_test, y_pred_lr, "Linear Regression")
evaluate_model(y_test, y_pred_dt, "Decision Tree")
evaluate_model(y_test, y_pred_rf, "Random Forest")
evaluate_model(y_test, y_pred_svr, "Support Vector Regressor")
evaluate_model(y_test, y_pred_knn, "K-Nearest Neighbors")
evaluate_model(y_test, y_pred_xgb, "XGBoost")


Linear Regression Results:
Mean Squared Error: 0.19354872209777108
R2 Score: 0.8910979492440814

Decision Tree Results:
Mean Squared Error: 1.8562749999999968
R2 Score: -0.04445098926990276

Random Forest Results:
Mean Squared Error: 0.5804748849999901
R2 Score: 0.6733902207999496

Support Vector Regressor Results:
Mean Squared Error: 0.2205386098505955
R2 Score: 0.8759118292113701

K-Nearest Neighbors Results:
Mean Squared Error: 0.46805200000000086
R2 Score: 0.7366460387443896

XGBoost Results:
Mean Squared Error: 1.0085496937375973
R2 Score: 0.4325298108566389


In [18]:
# Function to predict the next day's temperature using the best model (XGBoost)
def predict_next_day_weather():
    # Fetch the weather data
    df 

    # Define features (X) and target (y)
    X = df.drop(['temp'], axis=1)
    y = df['temp']

    # Feature scaling
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Train the XGBoost model using all available data
    model = XGBRegressor(n_estimators=100, random_state=42)
    model.fit(X_scaled, y)

    # Use the latest data point (most recent day) to predict the next day temperature
    last_day_data = X.iloc[-1:].values  # Take the last row of data
    last_day_data_scaled = scaler.transform(last_day_data)

    # Predict the temperature for the next day
    next_day_prediction = model.predict(last_day_data_scaled)
    
    return next_day_prediction[0]

# Predict next day temperature
predicted_temp = predict_next_day_weather()
print(f"Predicted temperature for the next day: {predicted_temp:.2f}°C")

Predicted temperature for the next day: 28.84°C


c:\Users\Deepraj\Desktop\universal-box\template\Data-Science\Prediction\Weather-Prediction\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
